In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
# -*- coding: utf-8 -*-
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from scipy import stats

#load data
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')


The data exploration part was inspired by this kernel: 
https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python
    

In [ ]:
#Only for numeric variables
df_train.describe()

In [ ]:
#scatter plot OverallQual/saleprice
var = 'OverallQual'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90);

****The boxplot shows that the price spread is larger when the Overall Quall is larger

In [ ]:
#box plot Neighborhood/saleprice
var = 'Neighborhood'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90);


****Some neighborhoods have higher average price than the others, for exemple: Nridght and StoneBr

In [ ]:
#correlation matrix
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
# GarageYrBlt and YearBuilt 
#TotalBsmtSF  1stFlrSF  2ndFlrSF*
#GarageArea and GarageArea
corrmat[abs(corrmat['SalePrice']>0.5)]


In [ ]:
#select varaible: OverallQual YearBuilt YearRemodAdd TotalBsmtSF GrLivArea FullBath TotRmsAbvGrd GarageCars
feature=['SalePrice', 'OverallQual', 'GrLivArea', 'YearRemodAdd','GarageCars', 'TotRmsAbvGrd', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
df_train_se=df_train[feature]
#missing data
total = df_train_se.isnull().sum().sort_values(ascending=False)
percent = (df_train_se.isnull().sum()/df_train_se.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
df_train_se = df_train_se.drop((missing_data[missing_data['Total'] > 1]).index,1)
df_train_se = df_train_se.drop(df_train.loc[df_train['Electrical'].isnull()].index)
df_train_se.isnull().sum().max() #just checking that there's no missing data missing...


In [ ]:
#applying log transformation: because sale price is not normal distribution
df_train_se['SalePrice']=np.log(df_train_se['SalePrice'])
#create column for new variable (one is enough because it's a binary categorical feature)
#if area>0 it gets 1, for area==0 it gets 0
def trans(inputdata, var1):
    inputdata['HasBsmt'] = pd.Series(len(inputdata[var1]), index=inputdata.index)
    inputdata['HasBsmt'] = 0 
    inputdata.loc[inputdata[var1]>0,'HasBsmt'] = 1
    inputdata.loc[inputdata['HasBsmt']==1,var1] = np.log(inputdata[var1])
    return inputdata

trans(df_train_se,'TotalBsmtSF')
trans(df_train_se,'GrLivArea')

In [ ]:
#transformed histogram and normal probability plot
def plot(var):
    sns.distplot(df_train_se[var], fit=norm);
    fig = plt.figure()
    res = stats.probplot(df_train_se[var], plot=plt)
    return (res)

plot('SalePrice')

In [ ]:
#histogram and normal probability plot
sns.distplot(df_train_se[df_train_se['TotalBsmtSF']>0]['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train_se[df_train_se['TotalBsmtSF']>0]['TotalBsmtSF'], plot=plt)


In [ ]:
feature0=['OverallQual', 'GrLivArea', 'YearRemodAdd','GarageCars', 'TotRmsAbvGrd', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
df_train_X=df_train_se[feature0]
df_train_Y= df_train_se['SalePrice']

In [ ]:
#1.Linear Regression
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
# Create linear regression object
regr = linear_model.LinearRegression()
# Train the model using the training sets
regr.fit(df_train_X, df_train_Y)

# Make predictions using the testing set
df_train_pred = regr.predict(df_train_X)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(df_train_Y, df_train_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(df_train_Y, df_train_pred))


In [ ]:
#2. Ridge regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
X_train, X_test, y_train, y_test = train_test_split(df_train_X, df_train_Y,
                                                   random_state = 0)

linridge = Ridge(alpha=20.0).fit(X_train, y_train)

print('House Price Prédiction')
print('ridge regression linear model intercept: {}'
     .format(linridge.intercept_))
print('ridge regression linear model coeff:\n{}'
     .format(linridge.coef_))
print('R-squared score (training): {:.3f}'
     .format(linridge.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linridge.score(X_test, y_test)))
print('Number of non-zero features: {}'
     .format(np.sum(linridge.coef_ != 0)))


In [ ]:
#3. Lasso regression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(df_train_X, df_train_Y,
                                                   random_state = 0)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

linlasso = Lasso(alpha=0.001, max_iter = 10000).fit(X_train, y_train)

print('Crime dataset')
print('lasso regression linear model intercept: {}'
     .format(linlasso.intercept_))
print('lasso regression linear model coeff:\n{}'
     .format(linlasso.coef_))
print('Non-zero features: {}'
     .format(np.sum(linlasso.coef_ != 0)))
print('R-squared score (training): {:.3f}'
     .format(linlasso.score(X_train, y_train)))
print('R-squared score (test): {:.3f}\n'
     .format(linlasso.score(X_test, y_test)))
print('Features with non-zero weight (sorted by absolute magnitude):')



In [ ]:
#4. GBM
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)
mse = mean_squared_error(y_test, clf.predict(X_test))
print("MSE: %.4f" % mse)
# Plot training deviance

# compute test set deviance
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)

for i, y_pred in enumerate(clf.staged_predict(X_test)):
    test_score[i] = clf.loss_(y_test, y_pred)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, clf.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')

# #############################################################################
# Plot feature importance
feature_importance = clf.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
feature_importance
plt.yticks(pos, X_train.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()
